In [2]:
import cv2
from ultralytics import YOLO
from collections import defaultdict


# Call model
model = YOLO("yolo11n.pt")


# Print Total number of classes
class_list = model.names
print(class_list)


# open the video files
cap = cv2.VideoCapture("4.mp4")


# Define line positions for counting
line_y_red = 350 # Red line position
line_y_blue = line_y_red + 50 # Blue line position

# Variables to store counting and tracking information
counted_ids_red_to_blue = set()
counted_ids_blue_to_red = set()

# Dictionaries to count objects by class for each direction
count_red_to_blue = defaultdict(int) # Moving downwords
count_blue_to_red = defaultdict(int) # Moving upwards

# State dictionaries to track which line was crossed first
crossed_red_first = {}
crossed_blue_first = {}


# Loop through video frames

while cap.isOpened():
    print("1")
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO tracking on the frame
    results = model.track(frame, persist = True, # bytetrack.yaml
                          tracker="bytetrack.yaml", # botsort.yaml
                           save = True)
    print(results)


    # Ensure results are not empty
    if results[0].boxes.data is not None:

      # Get the detected boxes, their class indices, and track IDs
      boxes = results[0].boxes.xyxy.cpu()
      track_ids = results[0].boxes.id.int().cpu().tolist()
      class_indices = results[0].boxes.cls.int().cpu().tolist()
      confidence = results[0].boxes.conf.cpu()

      # Draw the line of the each frame
      cv2.line(frame, (70, line_y_red), (1200, line_y_red ), (0, 0, 255), 3)
      cv2.putText(frame, "Red Line", (20, line_y_red - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

      cv2.line(frame, (70, line_y_blue), (1200, line_y_blue), (255, 0, 0), 3)
      cv2.putText(frame, "Blue Line", (20, line_y_blue - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

      # Loop through each detected object
      for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidence):
        x1, y1, x2, y2 = map(int, box)

        cx = (x1 + x2) // 2 # Calculate the center point
        cy = (y1 + y2) // 2


        # Get the class name using the class index
        class_name = class_list[class_idx]

        # Draw a dot at the center and display the tracking ID and class name
        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
        cv2.putText(frame, f"ID: {track_id} {class_name} {confidence[-1]:.2f}", (x1, y1 - 10),
        cv2.FONT_HERSHEY_COMPLEX, 0.6, (0, 255, 255), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)


        # check if the object crosse the red line
        if line_y_red - 3 <= cy <= line_y_red + 3:
          # Record that the object crossed the red line
          if track_id not in crossed_red_first:
            crossed_red_first[track_id] = True

        # check if the object crosses the blue line
        if line_y_blue - 3 <= cy <= line_y_blue + 3:
          # Record that the object crossed the blue line
          if track_id not in crossed_blue_first:
            crossed_blue_first[track_id] = True


        # Counting logic for downward direction (red -> blue)
        if track_id in crossed_red_first and track_id not in counted_ids_red_to_blue:
          if line_y_blue -5 <= cy <= line_y_blue + 5:
            count_red_to_blue[class_name] += 1
            counted_ids_red_to_blue.add(track_id)

        # Counting logic for upward direction (blue -> red)
        if track_id in crossed_blue_first and track_id not in counted_ids_blue_to_red:
          if line_y_red -5 <= cy <= line_y_red + 5:
            count_blue_to_red[class_name] += 1
            counted_ids_blue_to_red.add(track_id)

    # Display the counts on the frame
    y_offset = 30
    for class_name, count in count_red_to_blue.items():
      cv2.putText(frame, f"{class_name} (Down): {count}", (10, y_offset),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
      y_offset += 30

    y_offset += 20 # Add spacing for upward counts
    for class_name, count in count_blue_to_red.items():
      cv2.putText(frame, f"{class_name} (Up): {count}", (10, y_offset),
                 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0 ,255), 2, cv2.LINE_AA)
      y_offset += 30

     


    # Show the frame
    cv2.imshow("YOLO Object Tracking & Counting", frame)

    # Exit loop if 'ESC' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


RuntimeError: PytorchStreamReader failed reading zip archive: unsupported multidisk archive